# gridded_data_tutorial
## Notebook 3
Waterhackweek 2020
Steven Pestana (spestana@uw.edu)
***

### Study Case:

We want to look at precipitation data from DayMet for the Yakima watershed.

Also, I'm interested in snow too, so we'll look at that too.

### Research question:

Have the seasonal and inter-annual patterns of precipitation (and snow accumulaiton) in the Yakima watershed changed in recent years?

### Where is the Yakima watershed?

map figure

---
## Geospatial Raster Data and NetCDFs

Raster data is stored as a grid of values (hence it is also called "gridded" data). These can be visualized as pixels in a 2-dimensional image, where the color or brightness of each pixel represents the data value. 

Geospatial raster data represents scalar or categorical data distributed across Earth's surface, such as topography or land-use/landcover map. To make a raster "geospatial" it needs metadata that relates each pixel to a location on Earth's surface. **Coordinate reference systems** (CRS) specify the datum, projection, and other information to translate the raster pixel coordinates to locations on Earth. More details on coordinate reference systems can be found [here](https://datacarpentry.org/organization-geospatial/03-crs/index.html).

*raster example figures for scalar or categorical data*

Examples of raster data formats:
* geotiff ...
* HDF ...
* GRIB ...
* NetCDF ...

File formats such as [NetCDF](https://www.unidata.ucar.edu/software/netcdf/docs/) combine multi-dimensional data sets along with their metadata in a single file. 

---
## What is DayMet?

introduction to the dataset we'll be using



### How can we access this data? 

using OPeNDAP

* show how to search and select DayMet data using the OPeNDAP endpoint

In [ ]:
import xarray as xr

OPeNDAP endpoints for DayMet:

what data can we get and how do we select it?

In [ ]:
ds = xr.open_dataset("https://thredds.daac.ornl.gov/thredds/dodsC/daymet-v3-agg/na.ncml")

What did we just open with xarray? Look at the dataset's structure and metadata:

We have *Dimensions* of **time**, **x**, and **y**; *Coordinates* of **time**, **x**, **y**, **lat**, and **lon**.



In [ ]:
ds

### Data selecting/slicing/indexing methods

* define some lat/lon bounding boxes
* define some time bounds

To select a smaller area to work with, we can use the [*xarray.Dataset.isel*](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.isel.html) method. 

In [ ]:
testdata_srad = ds.srad.isel(time=100, x=slice(1000,3000), y=slice(1000,3000))

In [ ]:
testdata_srad.plot()

In [ ]:
testdata_srad.plot(x='lon', y='lat')

In [ ]:
# yay! we now know how to access this data source

Look at the metadata of what DayMet gives us

* introducing some new stuff using xarray

In [ ]:
# yay! we now know what the data structure is and what the metadata tells us

Try making some plots with this data now showing xarray plotting functions, cartopy

In [ ]:
# yay! we made some plots

Next section we'll use ulmo, xarray to answer our **research question**